# Parse LSDYNA file to extract particle coordinate, type, effective plastic strain

In [2]:
import re
import numpy as np

LOADING_PARTICLES = []
SUPPORT_PARTICLES = []

def parse_simulation(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    # Find all "particle position" lines and "plastic strain" lines using key words
    pos_lines_start, pos_lines_end = [], []
    strain_lines_start, strain_lines_end = [], []
    for idx, line in enumerate(lines):
        if line.startswith("*NODE"):
            pos_lines_start.append(idx)
        elif line.startswith("$NODAL_RESULTS"):  # $NODAL_RESULTS,(1d) *INITIAL_VELOCITY_NODE(2d)
            pos_lines_end.append(idx)
        elif line.startswith("$RESULT OF Effective Plastic Strain"):
            strain_lines_start.append(idx)
        elif line.startswith("*END"):  
            strain_lines_end.append(idx)
            
    # Extact particle positions 
    trajectory = []
    for line_start, line_end in zip(pos_lines_start, pos_lines_end):
        pos_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        timestep = []
        for line in pos_lines:
            num_str = re.findall(r'[-\d\.e+]+', line)  # Regular expression findign scitific numbers
            (x, y) = (float(num_str[1]), float(num_str[2]))
            timestep.append((x,y))
        timestep.append((x,y)) # Add one particle as an additional al boundary (as there are only 4 boundary particles but 5 concrete particles in one column)
        trajectory.append(timestep) 
    
    # Extact particle types
    particle_types = []
    pos_lines = lines[pos_lines_start[0]+1:pos_lines_end[0]]
    for line in pos_lines:
        num_str = re.findall(r'[-\d\.e+]+', line)
        if int(num_str[0]) in LOADING_PARTICLES:
            particle_types.append(3)   # kinematic particles
        elif int(num_str[0]) in SUPPORT_PARTICLES:
            particle_types.append(2)   # boundary particles (rigid)
        else:
            particle_types.append(1)   # normal concrete particles
    particle_types.append(2)  # add one boundar particle type
    # Extrac effective plastic strain
    strains = []
    for line_start, line_end in zip(strain_lines_start, strain_lines_end):
        strain_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        strains_one_step = []
        for line in strain_lines:
            num_str = re.findall(r'[-+\d\.Ee]+', line)  # the expression matches one or more repetitions of "-", "integer", ".", "E",
            num = float(num_str[1])
            strains_one_step.append(num)
        strains_one_step.append(num)   # add one more strain value for the additional boundary particle (no effect, just keep the shape uniform)
        strains.append(strains_one_step)     
    

    return np.array(trajectory).astype(np.float), np.array(particle_types).astype(np.float), np.array(strains).astype(np.float)

# Pre-process and write to npz for GNN training and testing

In [7]:
import glob
import json
import random
import numpy as np

dataset = 'Concrete1D'

in_dir = f'/home/jovyan/share/gns_data/{dataset}/LSDYNA/'
out_dir = f'/home/jovyan/share/gns_data/{dataset}/'
STEP_SIZE = 3

val_set = ['300_4', '400_8', '500_2']
test_set = [ '300_2', '400_4', '500_8']

## Normalisation parameters
pos_max, pos_min = np.array([500, 4]), np.array([-60, -4])
strain_min, strain_max = -0.18, 0.18
strain_mean, strain_std = 0.0035714929721944264, 0.045594130780353334

simulations = glob.glob(in_dir + '*')
random.shuffle(simulations)
simulations.sort()
ds_train, ds_valid, ds_test = {}, {}, {}
vels = np.array([]).reshape(0, 2)
accs = np.array([]).reshape(0, 2)
train_info, valid_info, test_info = [], [], []

for idx, simulation in enumerate(simulations):
    print(f"{idx} Reading {simulation}...")
    
    trajectory_name = simulation.split('/')[-1]
    positions, particle_types, strains = parse_simulation(simulation)
    
    # Preprocessing positions
    positions = positions[::STEP_SIZE,::1,::1] 
    positions[:, -5:, 1] = [-4, -2, 0, 2, 4]   ## Modify the y coordinate of the last 4 particles (boundary)
    positions = (positions - pos_min) / (pos_max - pos_min)  # Normalize based on overall min and max of all simulations
    # y_scalling_factor = (pos_max - pos_min)[0] / (pos_max - pos_min)[1]
    # positions[:,:,1] = positions[:,:,1] / y_scalling_factor   
    
    # Change the last 4 particle to boundary particle
    particle_types[-5:] = 2

    # Preprocessing strains
    strains = strains[::STEP_SIZE, ::1]
    # strains = (strains - strain_mean) / strain_std   ## standardize based on overall mean and std
    # strains = (strains - strain_min) / (strain_max - strain_min)   # Normalize based on overall min and max of all simulations
    
    print(f"Position min:{positions.min(axis=(0,1))}, max:{positions.max(axis=(0,1))}")
    print(f"Strain min:{strains.min(axis=(0,1))}, max:{strains.max(axis=(0,1))}")
    print(f"Position shape:{positions.shape}, type shape:{particle_types.shape}")
    print(f"Unique particle types: {np.unique(particle_types)}")
    
    # Data splits: train, valid, test
    key = 'trajectory_' + str(idx)
    if any(name in trajectory_name for name in val_set):
        ds_valid[key] = [positions, particle_types, strains]
        valid_info.append(trajectory_name)
    if any(name in trajectory_name for name in test_set):
        ds_test[key] = [positions, particle_types, strains]
        test_info.append(trajectory_name)
    else:
        ds_train[key] = [positions, particle_types, strains]   
        train_info.append(trajectory_name)
        
    # Extract Vel and Acc statistics
    # positions of shape [timestep, particles, dimensions]
    vel_trajectory = positions[1:,:,:] - positions[:-1,:,:]
    acc_trajectory = vel_trajectory[1:,:,:]- vel_trajectory[:-1,:,:]
    
    vels = np.concatenate((vels, vel_trajectory.reshape(-1, 2)), axis=0)
    accs = np.concatenate((accs, acc_trajectory.reshape(-1, 2)), axis=0)
    
vel_mean = list(vels.mean(axis=0))
vel_std = list(vels.std(axis=0))
acc_mean = list(accs.mean(axis=0))
acc_std = list(accs.std(axis=0))

np.savez(out_dir + 'train.npz', **ds_train)
np.savez(out_dir + 'valid.npz', **ds_valid)
np.savez(out_dir + 'test.npz', **ds_test)

print(f"{len(ds_train)} trajectories parsed and saved to train.npz.")
print(f"{len(ds_valid)} trajectories parsed and saved to valid.npz.")
print(f"{len(ds_test)}  trajectories parsed and saved to test.npz.")

print(f"Positions shape {positions.shape}, Particle types shape {particle_types.shape}, Strains shape {strains.shape}")

# Save meta data
in_file = '/home/jovyan/share/gns_data/WaterDropSample/metadata.json'
out_file = f'/home/jovyan/share/gns_data/{dataset}/metadata.json'

with open(in_file, 'r') as f:
    meta_data = json.load(f)

# meta_data['bounds'] = [[-200, 200], [0, 100]]
# The origin of simulation domain is at bottom center, and x in [-165, 165], y in [-10, 85].
# Particle radius r is 1.25 mm, and the connection Radius R is around 6r to 7r, or [7.5, 8.75] (24 neighbors, maybe more)
# In GNN, the suggested connection radius is 4.5r, or 5.625 mm (aounrd 20 neighbors)
# If R is 6mm before normalization, then it is 0.016 where 6/370 = x/1
meta_data['sequence_length'] = positions.shape[0]
meta_data['default_connectivity_radius'] = 0.03  # 0.04 (normalized) or 7.5 (unnormalized) for around 24 neighbours
meta_data['vel_mean'] = vel_mean
meta_data['vel_std'] = vel_std
meta_data['acc_mean'] = acc_mean
meta_data['acc_std'] = acc_std
meta_data['dim'] = 2
meta_data['dt'] = 0.001 * STEP_SIZE
meta_data['bounds'] = [[0, 1], [0, 1]]
meta_data['file_train'] = train_info
meta_data['file_valid'] = valid_info
meta_data['file_test'] = test_info
print(meta_data)

with open(out_file, 'w') as f:
    json.dump(meta_data, f)

0 Reading /home/jovyan/share/gns_data/Concrete1D/LSDYNA/1d_200_1.txt...
Position min:[ 0.10403911 -0.0005664 ], max:[0.46428571 1.0005664 ]
Strain min:-0.021641, max:0.0216253
Position shape:(101, 505, 2), type shape:(505,)
Unique particle types: [1. 2.]
1 Reading /home/jovyan/share/gns_data/Concrete1D/LSDYNA/1d_200_2.txt...
Position min:[ 0.09908228 -0.00124148], max:[0.46428571 1.00124148]
Strain min:-0.043257, max:0.0433605
Position shape:(101, 505, 2), type shape:(505,)
Unique particle types: [1. 2.]
2 Reading /home/jovyan/share/gns_data/Concrete1D/LSDYNA/1d_200_4.txt...
Position min:[ 0.08898621 -0.00238507], max:[0.46428571 1.00238507]
Strain min:-0.085851, max:0.0860487
Position shape:(101, 505, 2), type shape:(505,)
Unique particle types: [1. 2.]
3 Reading /home/jovyan/share/gns_data/Concrete1D/LSDYNA/1d_200_8.txt...
Position min:[ 0.06807272 -0.00438923], max:[0.46428571 1.00438923]
Strain min:-0.17014, max:0.170763
Position shape:(101, 505, 2), type shape:(505,)
Unique partic